<a href="https://colab.research.google.com/github/ZIKZAK772/Artificial_intelligence/blob/master/tile-world/TileWorld_genetic_network_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random

class TileWorldProblem:
    def __init__(self):
        self.initial_state = [[3, 6, 1],
                              [4, 8, 2],
                              [7, 0, 5]]
        self.goal_state = [[1, 2, 3],
                           [4, 5, 6],
                           [7, 8, 0]]
        self.moves = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # left, right, up, down

    def get_possible_moves(self, state):
        empty_pos = self.find_empty_tile(state)
        possible_moves = []
        for move in self.moves:
            new_pos = (empty_pos[0] + move[0], empty_pos[1] + move[1])
            if self.is_valid_move(new_pos):
                possible_moves.append(new_pos)
        return possible_moves

    def is_valid_move(self, position):
        return 0 <= position[0] < 3 and 0 <= position[1] < 3

    def find_empty_tile(self, state):
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    return (i, j)
        return None

    def apply_move(self, state, move):
        empty_pos = self.find_empty_tile(state)
        new_state = [row[:] for row in state]  # Create a copy of the state
        new_state[empty_pos[0]][empty_pos[1]] = new_state[move[0]][move[1]]
        new_state[move[0]][move[1]] = 0
        return new_state

    def evaluate_fitness(self, state):
        # Evaluate the fitness by counting the number of tiles in correct position
        fitness = 0
        for i in range(3):
            for j in range(3):
                if state[i][j] == self.goal_state[i][j]:
                    fitness += 1
        return fitness

    def generate_random_individual(self, num_moves):
        individual = []
        state = self.initial_state
        for _ in range(num_moves):
            possible_moves = self.get_possible_moves(state)
            move = random.choice(possible_moves)
            individual.append(move)
            state = self.apply_move(state, move)
        return individual

    def apply_individual(self, individual):
        state = self.initial_state
        for move in individual:
            state = self.apply_move(state, move)
        return state

    def solve(self, population_size, num_generations):
        population = [self.generate_random_individual(50) for _ in range(population_size)]
        for generation in range(num_generations):
            population = sorted(population, key=lambda x: self.evaluate_fitness(self.apply_individual(x)), reverse=True)
            if self.evaluate_fitness(self.apply_individual(population[0])) == 9:
                print("Solution found in generation:", generation)
                break
            parents = self.select_parents(population)
            population = self.reproduce(parents, population_size)
        return population[0]

    def select_parents(self, population):
        # Tournament selection
        parents = []
        for _ in range(len(population)):
            tournament = random.sample(population, 5)
            winner = max(tournament, key=lambda x: self.evaluate_fitness(self.apply_individual(x)))
            parents.append(winner)
        return parents

    def reproduce(self, parents, population_size):
        # Single point crossover and mutation
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            if random.random() < 0.1:
                mutation_point = random.randint(0, len(child) - 1)
                possible_moves = self.get_possible_moves(self.apply_individual(child))
                mutated_move = random.choice(possible_moves)
                child[mutation_point] = mutated_move
            new_population.append(child)
        return new_population

def print_state(state):
    for row in state:
        print(" ".join(map(str, row)))
    print()

def main():
    problem = TileWorldProblem()
    solution = problem.solve(population_size=300, num_generations=1000)
    print("Solution:")
    print_state(problem.apply_individual(solution))

if __name__ == "__main__":
    main()


Solution found in generation: 8
Solution:
1 2 3
4 5 6
7 8 0

